# Problem Statement:

1.visit the link:https://archive.ics.uci.edu/ml/datasets/Carbon+Nanotubes
2.download the dataset
3.insert bulk data(csv file in mongodb)
4.different operation.

4.1 insertiion
4.2 update
4.3 deletion
4.4 find operation
4.5 filter

code evalution

modular coding
execption handling
proper log


In [1]:
#importing modules#
import pymongo
import logging
import csv

In [2]:
logging.basicConfig(filename='carbon_nanotubes_logfiles.log',
                    filemode='a',
                    format='%(asctime)s-%(levelname)s-%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO
                   )

logger=logging.getLogger()
logger.setLevel(logging.DEBUG)
                    

In [3]:
class Mongobd:                    #Mongodb class with basic mongo operations#
    def __init__(self,client_url):
        self.client_url=client_url
        
        try:
            self.client=pymongo.MongoClient(self.client_url)
        except Exception as e:
            print("Connection issue", e)
            logger.error('Connection Issue: ' + str(e))
        else:
            print("Connection Established")
            logger.info('Connection Established!' + str(self.client))
            
            
    
    def create_database(self, db_name):
        
        try:
            self.database = self.client[str(db_name)]
        except Exception as e:
            print("Database Creation issue", e)
            logger.error('Database Creation issue: ' + str(e))
        else:
            print("Database Created")
            logger.info('Database ceated:' + db_name + str(self.database))
        
    
    def create_collection(self,collection_name):
        
        try:
            self.collection= self.database[str(collection_name)]
        except exception as e:
            print("Collection creation issue", e)
            logger.error('collection creation issue: ' + str(e))
        else:
            print("collection created")
            logger.info('collection created: ' + collection_name + str(self.collection))
            
    
    def insert(self,record):
        
        try:
            if type(record) == dict:
                self.collection.insert_one(record)
                record_count = 1
            elif type(record) == list:
                self.collection.insert_many(record)
                record_count = len(record)
        except Exception as e:
            print("Data insertion issue" , e)
            logger.error('Data Insertion issue:' + str(e))
        else:
            print("Data Inserted:{} records".format(record_count))
            logger.info('Data Inserted:' + str(record_count) + 'records')
            
    
    def update(self, present_data , new_data ,update_all = False):
        
        try:
            new_data ={'$set':new_data}
            if update_all == True:
                self.collection.update_many(present_data , new_data)
            else:
                self.collection.update_one(present_data,new_data)
        except Exception as e:
            print("Update failed" , e)
            logger.error('Update failed: ' + str(e))
        else:
            logger.info('Data Updated')
            
    
    def delete(self,delete_data,delete_all=False):
        
        try:
            if delete_all == True:
                self.collection.delete_many(delete_data)
            else:
                self.collection.delete_one(delete_data)
        except Exception as e:
            print("Delete failed" , e)
            logger.error('Delete failed:'+ str(e))
            
        else:
            print("Delete Done")
            logger.info('Record deleted')
            
    
    def find(self , query):
        logger.info('Finding records for query:' + str(query))
        return self.collection.find(query)
    
    
    def display(self , count):
        logger.info('Displaying the top:' + str(count) + 'records')
        return self.collection.find().limit(count)
      

# Creating Database and Collection


In [4]:
#creating cluster object of Mongodb class and Estblished connection with mongodb atlas#

cluster= Mongobd(client_url='mongodb+srv://<mongodbtask>:<mongodb>@cluster0.fblem.mongodb.net/myFirstDatabase?retryWrites=true&w=majority')

Connection Established


In [5]:
#creating database and collection

cluster.create_database('carbon_nano_db')
cluster.create_collection('nanotube_collections')

Database Created
collection created


# Task 3: Insert the bulk CSV (carbon_nanotube_data) 

In [6]:
with open('carbon_nanotubes.csv','r') as f:
    data = csv.reader(f , delimiter = '\n')
    itr = 0
    records = []
    
    for i in data:
        if itr == 0:
            itr += 1
            continue
        row_data = i[0].split(';')
        record={'m': int(row_data[0]),      # Chiral indice n
            'n': int(row_data[1]),      # Chiral indice m
            'u': (row_data[2]),           # Initial atomic coordinate u
            'v': (row_data[3]),           # Initial atomic coordinate v
            'w': (row_data[4]),           # Initial atomic coordinate w
            'u': (row_data[5]),          # Calculated atomic coordinates u'
            'v': (row_data[6]),          # Calculated atomic coordinates v'
            'w': (row_data[7]),         # Calculated atomic coordinates w'
        }
        records.append(record)

In [ ]:
cluster.insert(records)

# Task 4: Different MongoDB Operations

4.1 Data Insertion

In [ ]:
record = {
    'm': 112,
    'n': 2,
    'u': '0,35442',
    'v': '0,363635',
    'w': '0,3452562',
    "u'": '0,2122',
    "v'": '0,21556',
    "w'": '0,09834',
}
cluster.insert(record)

# 4.2 update

Updating only one record (Update_all = False)

In [ ]:
present_data = {'u': '0,111111'}
new_data = {'u': '0,323232', "u'": '0,323232', "v": '0,222222'}
cluster.update(present_data, new_data, update_all=False)

In [ ]:
cluster.update(present_data, new_data, update_all=True)

# 4.3 Delete Data

Deleting 1 record with m = 1 (delete_all = False)

In [ ]:
delete_data = {'m': 1}
cluster.delete(delete_data, delete_all=False)

# 4.4 Find Operator

In [ ]:
## Checking if record for m = 1 exits
query = {'m': '1'}

for i in cluster.find(query):
    print(i)
query = {'m': {'$lte': 2}}

for i in cluster.find(query):
    print(i)

In [ ]:
query = {'u': {'$in': ['0,287448', '0,489336', '0,665708']}}

for i in cluster.find(query):
    print(i)

# 4.5 Display

In [ ]:
display_count = 10
for idx, record in enumerate(cluster.display(display_count)):
    print(idx, record)